In [ ]:
!pip install --upgrade --no-cache-dir gdown

! gdown 1iIX-vkOv8eJrg4i8Fib5C2r6eFmRE89_ #WebNLG files
! unzip /content/webnlg.zip -d /content

webnlg_path = '/content/webnlg'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From: https://drive.google.com/uc?id=1iIX-vkOv8eJrg4i8Fib5C2r6eFmRE89_
To: /content/webnlg.zip
100% 2.69M/2.69M [00:00<00:00, 207MB/s]
Archive:  /content/webnlg.zip
   creating: /content/webnlg/
  inflating: /content/webnlg/dataset_dict.json  
   creating: /content/webnlg/dev/
  inflating: /content/webnlg/dev/dataset.arrow  
  inflating: /content/webnlg/dev/dataset_info.json  
  inflating: /content/webnlg/dev/state.json  
   creating: /content/webnlg/test/
  inflating: /content/webnlg/test/dataset.arrow  
  inflating: /content/webnlg/test/dataset_info.json  
  inflating: /content/webnlg/test/state.json  
   creating: /content/webnlg/train/
  inflating: /content/webnlg/train/dataset.arrow  
  inflating: /content/webnlg/train/data

### Import WebNLG triples (for testing)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_from_disk

webnlg = load_from_disk(webnlg_path)

## Seen/Unseen Triples

In [ ]:
def load_webnlg(webnlg, set):
    dataset = []
    for sample in webnlg[set]:
        if sample['size'] == 1:
            dataset.append([sample['modified_triple_sets']['mtriple_set'][0][0], sample['lex']['text']])
    return dataset

In [ ]:
def merge_repeated_triples(dataset):

    triples = [] #create a list of triples to then find the repeated ones
    for sample in dataset:
        triples.append(sample[0])

    duplicates = [triple for triple in triples if triples.count(triple) > 1] #duplicated triples list

    to_delete = []
    for i, sample in enumerate(dataset): #Append sentences from repeated triples in the first occurrence triple list
        triple = sample[0]
        if triple in duplicates:
            first_occurrence = triples.index(triple)
            if i != first_occurrence:
                for sentence in sample[1]:
                    dataset[first_occurrence][1].append(sentence)
                to_delete.append(i) #Save indexes of triples to delete

    for i, index in enumerate(to_delete): #Delete repeated triples
        del dataset[index - i]

    for sample in dataset: #Remove repeated sentences after merging
        sentence_set = set(sample[1])
        sample[1] = list(sentence_set)

    return dataset

In [ ]:
#TEXT TRIPLES. Each triple is a string in this form: "Subject | Property | Object"

def get_text_triples(dataset):
    text_triples = []
    for sample in dataset:
        text_triples.append(sample[0])
    return text_triples

In [ ]:
#TRIPLES. Each triple is a list of strings in the form: ["Subject", "Property", "Object"]

def get_triples(text_triples):
    triples = []
    for text_triple in text_triples:
        firstBarIndex = text_triple.find('|')-1
        secondBarIndex = text_triple.rfind('|')+2

        sub = text_triple[:firstBarIndex]
        prop = text_triple[firstBarIndex + 3 : secondBarIndex-3]
        obj = text_triple[secondBarIndex:]

        triple = [sub, prop, obj]
        triples.append(triple)
    return triples

In [ ]:
def get_seenProperties():
    train = load_webnlg(webnlg, 'train')
    train = merge_repeated_triples(train)
    train_textTriples = get_text_triples(train)
    train_triples = get_triples(train_textTriples)

    properties = []
    for triple in train_triples:
        properties.append(triple[1])
    properties = list(set(properties))
    print('Seen Properties:', len(properties))
    return properties

In [ ]:
def get_unseenProperties(seenProperties):
    dev = load_webnlg(webnlg, 'dev')
    dev = merge_repeated_triples(dev)
    dev_textTriples = get_text_triples(dev)
    dev_triples = get_triples(dev_textTriples)
    dev_unseenProperties = []
    for triple in dev_triples:
        if triple[1] not in seenProperties:
            dev_unseenProperties.append(triple[1])
    dev_unseenProperties = list(set(dev_unseenProperties))
    print('Dev Unseen Properties:', len(dev_unseenProperties))

    test = load_webnlg(webnlg, 'test')
    test = merge_repeated_triples(test)
    test_textTriples = get_text_triples(test)
    test_triples = get_triples(test_textTriples)
    test_unseenProperties = []
    for triple in test_triples:
        if triple[1] not in seenProperties:
            test_unseenProperties.append(triple[1])
    test_unseenProperties = list(set(test_unseenProperties))
    print('Test Unseen Properties:', len(test_unseenProperties))

    unseenProperties = list(set(dev_unseenProperties + test_unseenProperties))
    print('All Unseen Properties:', len(unseenProperties))

    return unseenProperties

In [ ]:
seenProperties = get_seenProperties()

Seen Properties: 346


In [ ]:
unseenProperties = get_unseenProperties(seenProperties)

Dev Unseen Properties: 14
Test Unseen Properties: 45
All Unseen Properties: 54


In [ ]:
def get_dev_seenUnseenTriples(seen_unseen_Properties):
    dev = load_webnlg(webnlg, 'dev')
    dev = merge_repeated_triples(dev)
    dev_textTriples = get_text_triples(dev)
    dev_triples = get_triples(dev_textTriples)
    dev_seenUnseenTriples = []
    for triple in dev_triples:
        if triple[1] in seen_unseen_Properties:
            dev_seenUnseenTriples.append(triple)
    return dev_seenUnseenTriples

def get_test_seenUnseenTriples(seen_unseen_Properties):
    test = load_webnlg(webnlg, 'test')
    test = merge_repeated_triples(test)
    test_textTriples = get_text_triples(test)
    test_triples = get_triples(test_textTriples)
    test_seenUnseenTriples = []
    for triple in test_triples:
        if triple[1] in seen_unseen_Properties:
            test_seenUnseenTriples.append(triple)
    return test_seenUnseenTriples

In [ ]:
def get_seen_unseen_Triples(seen_unseen_Properties):
    dev_seenUnseenTriples = get_dev_seenUnseenTriples(seen_unseen_Properties)

    seenUnseenTriples = dev_seenUnseenTriples.copy()

    test_seenUnseenTriples = get_test_seenUnseenTriples(seen_unseen_Properties)

    for triple in test_seenUnseenTriples:
        if triple not in dev_seenUnseenTriples:
            seenUnseenTriples.append(triple)
    #print('Total Seen Triples:', len(seen_triples))
    #print('Total Unseen Triples:', len(seen_triples))

    return seenUnseenTriples

In [ ]:
dev_seenTriples = get_dev_seenUnseenTriples(seenProperties)
len(dev_seenTriples)

387

In [ ]:
seenTriples = get_seen_unseen_Triples(seenProperties)
len(seenTriples)

795

In [ ]:
unseenTriples = get_seen_unseen_Triples(unseenProperties)
len(unseenTriples)

127

## Selecting triples

In [ ]:
#Selected for the first experiment. From seen triples (from the webnlg dev modified triples of size 1).

chosenGood_triples = [['Ace_Wilder', 'birthDate', '1982-07-23'],
['Albennie_Jones', 'birthPlace', 'Errata,_Mississippi'],
['United_Kingdom', 'capital', 'London'],
['Alderney_Airport', 'cityServed', 'Alderney'],
['Dublin', 'country', 'Republic_of_Ireland'],
['Allen_Forrest', 'genre', 'Pop_music'],
['Amarillo,_Texas', 'isPartOf', 'United_States'],
['Alberto_Teisaire', 'nationality', 'Argentina'],
['Philippines', 'language', 'Arabic'],
['Asser_Levy_Public_Baths', 'yearOfConstruction', '1904']]


In [ ]:
#Selected for the second experiment (only 20 random will be used). From seen triples (from the webnlg dev modified triples of size 1)

good_unseenTriples = [['Imagine_(John_Lennon_song)', 'album', 'Imagine_(John_Lennon_album)'],
 ['Mermaid_(Train_song)', 'album', 'California_37_(Train_album)'],
 ['Istanbul', 'areaMetro', '5343000000.0'],
 ['Turn_Me_On_(album)', 'artist', 'The_Honeymoon_Killers_(American_band)'],
 ['Expect_a_Miracle', 'artist', 'Brian_Kelly_(composer_&_pianist)'],
 ['Nord_(Year_of_No_Light_album)', 'artist', 'Year_of_No_Light'],
 ['Live_at_Roadburn_2008_(Year_of_No_Light_album)',
  'artist',
  'Year_of_No_Light'],
 ['Bootleg_Series_Volume_1:_The_Quine_Tapes',
  'artist',
  'The_Velvet_Underground'],
 ['Death_on_a_Factory_Farm', 'broadcastedBy', 'HBO'],
 ['Super_Capers', 'budget', '2000000.0'],
 ['South_Jersey_Transportation_Authority', 'chief', 'Stephen_Dilts'],
 ['Bill_Oddie', 'child', 'Kate_Hardie'],
 ['John_Mills', 'child', 'Juliet_Mills'],
 ['John_Mills', 'child', 'Hayley_Mills'],
 ['Darinka_Dentcheva', 'citizenship', 'United_States'],
 ['Nurhan_Atasoy', 'citizenship', 'Turkey'],
 ['Kingdom_of_England', 'dissolutionYear', '1707'],
 ['Pontiac', 'extinctionDate', '2010-10-31'],
 ['Bananaman', 'firstAired', '"1983-10-03"'],
 ['The_Velvet_Underground', 'formerBandMember', 'Lou_Reed'],
 ['Train_(band)', 'formerBandMember', 'Charlie_Colin'],
 ['Train_(band)', 'formerBandMember', 'Johnny_Colt'],
 ['Roadside_Attractions', 'foundingYear', '2003-01-01'],
 ['MotorSport_Vision', 'foundingYear', '2004'],
 ['Brandon_Carter', 'knownFor', 'Anthropic_principle'],
 ['Brandon_Carter', 'knownFor', 'Carter_constant'],
 ['Brandon_Carter', 'knownFor', 'Doomsday_argument'],
 ['Darinka_Dentcheva', 'knownFor', 'Stochastic_programming'],
 ['Olga_Bondareva', 'knownFor', 'Bondareva–Shapley_theorem'],
 ['Andrzej_Piotr_Ruszczyński', 'knownFor', 'Stochastic_programming'],
 ['Brandon_Carter', 'knownFor', 'No-hair_theorem'],
 ['Brandon_Carter', 'knownFor', 'Penrose_diagram'],
 ['English_Without_Tears', 'musicComposer', 'Nicholas_Brodszky'],
 ["It's_Great_to_Be_Young_(1956_film)", 'musicComposer', 'Louis_Levy'],
 ['Death_on_a_Factory_Farm', 'musicComposer', 'Jamie_Lawrence'],
 ['Imagine_(John_Lennon_song)', 'musicalArtist', 'John_Lennon'],
 ['Mermaid_(Train_song)', 'musicalBand', 'Train_(band)'],
 ['Ciudad_Ayala', 'populationMetro', '1777539'],
 ['Istanbul', 'populationMetroDensity', '2691.0'],
 ['English_Without_Tears', 'producer', 'Anatole_de_Grunwald'],
 ['Take_It_Off!', 'producer', 'The_Honeymoon_Killers_(American_band)'],
 ['McVeagh_of_the_South_Seas',
  'producer',
  'The_Progressive_Motion_Picture_Company'],
 ['Take_It_Off!', 'producer', 'Wharton_Tiers'],
 ['Turn_Me_On_(album)', 'producer', 'The_Honeymoon_Killers_(American_band)'],
 ['Bootleg_Series_Volume_1:_The_Quine_Tapes',
  'producer',
  'The_Velvet_Underground'],
 ['Death_on_a_Factory_Farm', 'producer', 'Tom_Simon'],
 ['Imagine_(John_Lennon_song)', 'producer', 'Phil_Spector'],
 ['Squeeze_(The_Velvet_Underground_album)',
  'producer',
  'The_Velvet_Underground'],
 ['Live_at_Roadburn_2008_(Year_of_No_Light_album)',
  'producer',
  'Roadburn_Festival'],
 ['Death_on_a_Factory_Farm', 'producer', 'Sarah_Teale'],
 ['Mermaid_(Train_song)', 'producer', 'Espionage_(production_team)'],
 ['Turn_Me_On_(album)', 'producer', 'Wharton_Tiers'],
 ['Expect_a_Miracle', 'producer', 'Brian_Kelly_(composer_&_pianist)'],
 ['Nord_(Year_of_No_Light_album)', 'producer', 'Year_of_No_Light'],
 ['Abarth_1000_GT_Coupé', 'productionEndYear', '1958'],
 ['Pontiac_Rageous', 'productionEndYear', '1997'],
 ['Live_at_Roadburn_2008_(Year_of_No_Light_album)',
  'recordDate',
  '2008-04-19'],
 ['Mermaid_(Train_song)', 'releaseDate', '2012-12-27'],
 ['The_Fellowship_of_the_Ring', 'releaseDate', '1954-07-29'],
 ['Nord_(Year_of_No_Light_album)', 'releaseDate', '2006-09-06'],
 ['Bootleg_Series_Volume_1:_The_Quine_Tapes', 'releaseDate', '2001-10-16'],
 ['English_Without_Tears', 'releaseDate', '1944-07-28'],
 ['The_Hobbit', 'releaseDate', '1937-09-21'],
 ['Baymax', 'series', 'Big_Hero_6_(film)'],
 ['Airey_Neave', 'serviceStartYear', '1935'],
 ['University_of_Burgundy', 'staff', '2900'],
 ['Liselotte_Grschebina', 'training', 'School_of_Applied_Arts_in_Stuttgart'],
 ['University_of_Cambridge', 'viceChancellor', 'Leszek_Borysiewicz'],
 ["It's_Great_to_Be_Young_(1956_film)", 'writer', 'Ted_Willis'],
 ['Mermaid_(Train_song)', 'writer', 'Stargate_(production_team)'],
 ['Super_Capers', 'writer', 'Ray_Griggs_(director)'],
 ['McVeagh_of_the_South_Seas', 'writer', 'Harry_Carey_(actor_born_1878)'],
 ['English_Without_Tears', 'writer', 'Terence_Rattigan'],
 ['Mermaid_(Train_song)', 'writer', 'Amund_Bjørklund'],
 ['Mermaid_(Train_song)', 'writer', 'Pat_Monahan'],
 ['English_Without_Tears', 'writer', 'Anatole_de_Grunwald'],
 ['Mermaid_(Train_song)', 'writer', 'Espen_Lind']]

In [ ]:
len(unseenTriples), len(good_unseenTriples)

(127, 77)

In [ ]:
#For the second experiment (only 20 random will be used). From seen triples (from the webnlg dev modified triples of size 1)

good_seenTriples = [['Acta_Mathematica_Hungarica', 'academicDiscipline', 'Mathematics'],
['Alvah_Sabin', 'activeYearsEndDate', '1857-03-03'],
['AIDAluna', 'activeYearsStartDate', '2009-03-22'],
['Aaron_Bertram', 'activeYearsStartYear', '1998'],
["Alison_O'Donnell", 'activeYearsStartYear', '1963'],
['Abdul_Taib_Mahmud', 'almaMater', 'University_of_Adelaide'],
['Adenan_Satem', 'almaMater', 'University_of_Adelaide'],
['Josef_Klaus', 'almaMater', 'University_of_Vienna'],
['103_Colmore_Row', 'architect', 'John_Madin'],
['Amdavad_ni_Gufa', 'architect', 'B._V._Doshi'],
['Albany,_Oregon', 'areaTotal', '45.97 (square kilometres)'],
['La_Crosse,_Wisconsin', 'areaTotal', '58.38'],
['Aaron_Deer', 'associatedBand/associatedMusicalArtist', 'The_Horns_of_Happiness'],
['Abradab', 'associatedBand/associatedMusicalArtist', 'Magik_(rapper)'],
['Alan_Frew', 'associatedBand/associatedMusicalArtist', 'Glass_Tiger'],
['Aleksandra_Kovač', 'associatedBand/associatedMusicalArtist', 'Kornelije_Kovač'],
['Alfredo_Zitarrosa', 'associatedBand/associatedMusicalArtist', 'Hilario_Pérez'],
['Andrew_Rayel', 'associatedBand/associatedMusicalArtist', 'Jwaydan_Moyine'],
['1634:_The_Bavarian_Crisis', 'author', 'Eric_Flint'],
['A_Severed_Wasp', 'author', "Madeleine_L'Engle"],
['Wolf_Solent', 'author', 'John_Cowper_Powys'],
['Bill_Everett', 'award', 'Eisner_Award'],
['John_Buscema', 'award', 'Eagle_Award_(comics)'],
['Marv_Wolfman', 'award', 'Eagle_Award_(comics)'],
['Aaron_S._Daggett', 'award', 'Purple_Heart'],
['Adam_Koc', 'award', 'Cross_of_Valour_(Poland)'],
['Airey_Neave', 'award', 'Territorial_Decoration'],
['Aaron_S._Daggett', 'battle', 'Battle_of_Mine_Run'],
['Abner_W._Sibal', 'battle', 'World_War_II'],
['Alfred_Moore_Scales', 'battle', 'Battle_of_Chancellorsville'],
['Alfred_Moore_Scales', 'battle', 'Overland_Campaign'],
['Allan_Shivers', 'battle', 'World_War_II'],
['Ace_Wilder', 'birthDate', '1982-07-23'],
['Albennie_Jones', 'birthPlace', 'Errata,_Mississippi'],
['Andrew_Rayel', 'birthPlace', '"Chișinău, Moldova"'],
['Aaron_Boogaard', 'birthPlace', 'Canada'],
['Adam_Maher', 'birthPlace', '"Ait Ikkou, Morocco"'],
['Adam_Maher', 'birthPlace', 'Diemen'],
['Adam_McQuaid', 'birthPlace', 'Charlottetown'],
['Ahmad_Kadhim_Assad', 'birthPlace', 'Iraq'],
['Alaa_Abdul-Zahra', 'birthPlace', 'Baghdad'],
['Aleksander_Barkov,_Jr.', 'birthPlace', 'Finland'],
['Alex_Plante', 'birthPlace', 'Manitoba'],
['Airey_Neave', 'birthPlace', '"Knightsbridge, London"'],
['Denmark', 'capital', 'Copenhagen'],
['South_Africa', 'capital', 'Cape_Town'],
['United_Kingdom', 'capital', 'London'],
['Indiana', 'capital', 'Indianapolis'],
['New_Jersey', 'capital', 'Trenton,_New_Jersey'],
['A.C._Milan', 'chairman', 'Silvio_Berlusconi'],
['Premier_Development_League', 'champions', 'K-W_United_FC'],
['Columbus_Blue_Jackets', 'city', 'Columbus,_Ohio'],
['Alderney_Airport', 'cityServed', 'Alderney'],
['Amsterdam_Airport_Schiphol', 'cityServed', 'Amsterdam'],
['Andrews_County_Airport', 'cityServed', 'Andrews,_Texas'],
['Angola_International_Airport', 'cityServed', 'Luanda'],
['Abel_Hernández', 'club', 'Uruguay_Olympic_football_team'],
['Abel_Hernández', 'club', 'Uruguay_national_football_team'],
['Abner_(footballer)', 'club', 'C.D._FAS'],
['Ahmad_Kadhim_Assad', 'club', 'PAS_Tehran_F.C.'],
['Akeem_Priestley', 'club', 'FK_Mughan'],
['Alan_Martin_(footballer)', 'club', 'Accrington_Stanley_F.C.'],
['Alan_Martin_(footballer)', 'club', 'Crewe_Alexandra_F.C.'],
['Aleksander_Barkov,_Jr.', 'club', 'Florida_Panthers'],
['Aleksandre_Guruli', 'club', 'FC_Samtredia'],
['Gus_Poyet', 'club', 'AEK_Athens_F.C.'],
['Jens_Härtel', 'club', 'SV_Germania_Schöneiche'],
['John_van_den_Brom', 'club', 'AZ_Alkmaar'],
['Jorge_Humberto_Rodríguez', 'club', 'A.D._Isidro_Metapán'],
['Massimo_Drago', 'club', 'Delfino_Pescara_1936'],
['Massimo_Drago', 'club', 'S.S._Chieti_Calcio'],
['Stuart_Parker_(footballer)', 'club', 'Bury_F.C.'],
['Maccabi_Tel_Aviv_B.C.', 'coach', 'Žan_Tabak'],
['Nigerian_Air_Force', 'commander', 'Sadique_Abubakar'],
['11_Diagonal_Street', 'completionDate', '1983'],
['200_Public_Square', 'completionDate', '1985'],
['San_Sebastián_de_los_Reyes', 'country', 'Spain'],
['Budapest', 'country', 'Hungary'],
['Point_Fortin', 'country', 'Trinidad'],
['Adare_Manor', 'country', 'Republic_of_Ireland'],
['Ampara_Hospital', 'country', 'Sri_Lanka'],
['Dublin', 'country', 'Republic_of_Ireland'],
['Albany,_Oregon', 'country', 'United_States'],
['Amarillo,_Texas', 'country', 'United_States'],
['Atlantic_County,_New_Jersey', 'country', 'United_States'],
['California', 'country', 'United_States'],
['Georgia_(U.S._state)', 'country', 'United_States'],
['Tarrant_County,_Texas', 'country', 'United_States'],
['Asam_pedas', 'country', 'Malaysia'],
['Batchoy', 'country', 'Philippines'],
['Alba_Iulia', 'country', 'Romania'],
['School of Business and Social Sciences at the Aarhus University', 'country', 'Denmark'],
['Asterix_(comicsCharacter)', 'creator', 'René_Goscinny'],
['Auron_(comicsCharacter)', 'creator', 'Marv_Wolfman'],
['Balder_(comicsCharacter)', 'creator', 'Jack_Kirby'],
['Bolt_(comicsCharacter)', 'creator', 'Dan_Mishkin'],
['Barny_Cakes', 'creator', 'Mondelez_International'],
['Alfred_Garth_Jones', 'deathPlace', 'London'],
['Alan_Shepard', 'deathPlace', 'California'],
['Adam_Koc', 'deathPlace', 'New_York_City'],
['Edmund_J._Davis', 'deathPlace', 'Austin,_Texas'],
['Atatürk_Monument_(İzmir)', 'designer', 'Pietro_Canonica'],
['107_Camilla', 'discoverer', 'F._Vilas'],
['107_Camilla', 'discoverer', 'N._R._Pogson'],
['Adirondack_Regional_Airport', 'elevationAboveTheSeaLevel', '507'],
['Al-Taqaddum_Air_Base', 'elevationAboveTheSeaLevel', '84.0'],
['Antioch,_California', 'elevationAboveTheSeaLevel', '13.0'],
['Arlington,_Texas', 'elevationAboveTheSeaLevel', '184.0'],
['Antares_(rocket)', 'finalFlight', '2014-10-28'],
['Atlas_II', 'finalFlight', '1998-03-16'],
['300_North_LaSalle', 'floorCount', '60'],
['1634:_The_Ram_Rebellion', 'followedBy', '1635:_The_Cannon_Law'],
['Alcatraz_Versus_the_Evil_Librarians', 'followedBy', "Alcatraz_Versus_the_Scrivener's_Bones"],
['Castle_(novel)', 'followedBy', 'Aenir'],
['Caterpillar_Inc.', 'foundationPlace', 'California'],
['Florida_Panthers', 'generalManager', 'Dale_Tallon'],
['Aaron_Turner', 'genre', 'Avant-garde_metal'],
['Aaron_Turner', 'genre', 'Black_metal'],
['Aleksandra_Kovač', 'genre', 'Soul_music'],
['Alfredo_Zitarrosa', 'genre', 'Milonga_(music)'],
['Allen_Forrest', 'genre', 'Pop_music'],
['Andrew_White_(musician)', 'genre', 'Alternative_rock'],
['Twilight_(band)', 'genre', 'Black_metal'],
['Avocado', 'genus', 'Persea'],
['HIV', 'genus', 'Lentivirus'],
['FC_Terek_Grozny', 'ground', 'Grozny'],
['Olympique_Lyonnais', 'ground', 'Parc_Olympique_Lyonnais'],
['A.F.C._Fylde', 'ground', 'The_Fylde'],
['A.S._Gubbio_1910', 'ground', 'Italy'],
['Akron_Summit_Assault', 'ground', 'St._Vincent–St._Mary_High_School'],
['Port_Authority_of_New_York_and_New_Jersey', 'headquarter', 'Four_World_Trade_Center'],
['Guiana_Space_Centre', 'headquarter', 'Kourou,_French_Guiana'],
['Alex_Plante', 'height', '1.9304'],
['Akita_Museum_of_Art', 'inaugurationDate', '"2013-09-28"'],
['Harrietstown,_New_York', 'isPartOf', 'United_States'],
['Paracuellos_de_Jarama', 'isPartOf', 'Community_of_Madrid'],
['Saranac_Lake,_New_York', 'isPartOf', 'United_States'],
['Asunción', 'isPartOf', 'Gran_Asunción'],
['Chicago', 'isPartOf', 'DuPage_County,_Illinois'],
['Dublin', 'isPartOf', 'Leinster'],
['Abilene,_Texas', 'isPartOf', 'Texas'],
['Amarillo,_Texas', 'isPartOf', 'Potter_County,_Texas'],
['Amarillo,_Texas', 'isPartOf', 'United_States'],
['Anderson,_Indiana', 'isPartOf', '"Adams, Fall Creek, Lafayette, Richland, Union"'],
['Angola,_Indiana', 'isPartOf', 'Pleasant_Township,_Steuben_County,_Indiana'],
['Attica,_Indiana', 'isPartOf', 'Fountain_County,_Indiana'],
['Auburn,_Alabama', 'isPartOf', 'Alabama'],
['Swords,_Dublin', 'isPartOf', 'Dublin_(European_Parliament_constituency)'],
['Alba_Iulia', 'isPartOf', 'Alba_County'],
['Battle_of_Fredericksburg', 'isPartOfMilitaryConflict', 'American_Civil_War'],
['Belgium', 'language', 'German_language'],
['Greece', 'language', 'Greek_language'],
['Canada', 'language', 'Inuktitut'],
['Republic_of_Ireland', 'language', 'English_language'],
['Mexico', 'language', 'Spanish_language'],
['Philippines', 'language', 'Arabic'],
['Finland', 'language', 'Sami_languages'],
['Austria', 'language', 'Austrian_German'],
['Poland', 'language', 'Kashubian_language'],
['A_Wizard_of_Mars', 'language', 'English_language'],
['Delta_II', 'launchSite', 'Cape_Canaveral_Air_Force_Station'],
['Aarhus', 'leader', 'Jacob_Bundsgaard'],
['Belgium', 'leader', 'Philippe_of_Belgium'],
['Denmark', 'leader', 'Lars_Løkke_Rasmussen'],
['Flemish_Region', 'leader', 'Flemish_Government'],
['Greece', 'leader', 'Nikos_Voutsis'],
['Pakistan', 'leader', 'Anwar_Zaheer_Jamali'],
['Brandon,_Manitoba', 'leader', 'Drew_Caldwell'],
['India', 'leader', 'Sumitra_Mahajan'],
['Japan', 'leader', 'Tarō_Asō'],
['United_States', 'leader', 'Paul_Ryan'],
['Brazil', 'leader', 'Cármen_Lúcia'],
['Philippines', 'leader', 'Rodrigo_Duterte'],
['Derbyshire_Dales', 'leader', 'Patrick_McLoughlin'],
['Indonesia', 'leader', 'Jusuf_Kalla'],
['Germany', 'leader', 'Stanislaw_Tillich'],
['Azerbaijan', 'leader', 'Artur_Rasizade'],
['Argentina', 'leader', 'Gabriela_Michetti'],
['United_Kingdom', 'leader', 'Elizabeth_II'],
['United_States', 'leader', 'Joe_Biden'],
['Albany,_Georgia', 'leaderTitle', 'Mayor'],
['Albuquerque,_New_Mexico', 'leaderTitle', '"State Senate"'],
['Albuquerque,_New_Mexico', 'leaderTitle', 'Albuquerque_City_Council'],
['Ann_Arbor,_Michigan', 'leaderTitle', '"City Administrator"'],
['Ann_Arbor,_Michigan', 'leaderTitle', 'Mayor'],
['Swords,_Dublin', 'leaderTitle', '"County Manager"'],
['Al-Khor_Sports_Club', 'league', 'Qatar_Stars_League'],
['Aleksander_Barkov,_Jr.', 'league', 'National_Hockey_League'],
['FC_Tom_Tomsk', 'league', 'Russian_Football_National_League'],
['Adirondack_Regional_Airport', 'location', 'Harrietstown,_New_York'],
['Adolfo_Suárez_Madrid–Barajas_Airport', 'location', 'San_Sebastián_de_los_Reyes'],
['Agra_Airport', 'location', 'Agra'],
['Ardmore_Airport_(New_Zealand)', 'location', 'Auckland'],
['Athens_International_Airport', 'location', 'Spata'],
['Deram_Records', 'location', 'London'],
['103_Colmore_Row', 'location', 'Colmore_Row'],
['20_Fenchurch_Street', 'location', 'United_Kingdom'],
['3Arena', 'location', '"East Link Bridge"'],
['AIDA_Cruises', 'location', 'Rostock'],
['Caterpillar_Inc.', 'location', 'Peoria,_Illinois'],
['Meyer_Werft', 'location', 'Germany'],
['Meyer_Werft', 'location', 'Papenburg'],
["Baku_Turkish_Martyrs'_Memorial", 'location', 'Azerbaijan'],
['AZAL_PFK', 'location', '"Shuvalan, Baku, Azerbaijan"'],
['Olympic_Stadium_(Athens)', 'location', 'Marousi'],
['FC_Torpedo_Moscow', 'manager', 'Valery_Petrakov'],
['Peñarol', 'manager', 'Jorge_Orosmán_da_Silva'],
['1._FC_Köln', 'manager', 'Peter_Stöger'],
['AFC_Ajax', 'manager', 'Frank_de_Boer'],
['Agremiação_Sportiva_Arapiraquense', 'manager', 'Vica'],
['ALV_X-1', 'manufacturer', 'Alliant_Techsystems'],
['Ariane_5', 'manufacturer', 'Airbus_Defence_and_Space'],
['Athens', 'mayor', 'Giorgos_Kaminis'],
['Hip_hop_music', 'musicSubgenre', 'Gangsta_rap'],
['Buzz_Aldrin', 'nationality', 'United_States'],
['N._R._Pogson', 'nationality', 'England'],
['Alberto_Teisaire', 'nationality', 'Argentina'],
['Chinabank', 'netIncome', '15100000000'],
['John_Cowper_Powys', 'notableWork', 'Wolf_Solent'],
['A.F.C._Blackpool', 'numberOfMembers', '1500'],
['A.S._Livorno_Calcio', 'numberOfMembers', '19238'],
['AFC_Ajax_(amateurs)', 'numberOfMembers', '5000'],
['AZ_Alkmaar', 'numberOfMembers', '17023'],
['Akron_Summit_Assault', 'numberOfMembers', '3000'],
['Elliot_See', 'occupation', 'Test_pilot'],
['Agnes_Kant', 'office', 'House_of_Representatives_(Netherlands)'],
['Afonso_Pena_International_Airport', 'operatingOrganisation', 'Infraero'],
['Apollo_14', 'operator', 'NASA'],
['Apollo_8', 'operator', 'NASA'],
['101_Helena', 'orbitalPeriod', '1516.6700000000003 (days)'],
['10_Hygiea', 'orbitalPeriod', '5.57 (days)'],
['Aaron_Turner', 'origin', 'Boston'],
['Philippine_Entertainment_Portal', 'parentCompany', 'GMA_New_Media'],
['Amsterdam', 'part', 'Amsterdam-Centrum'],
['Alvah_Sabin', 'party', 'Whig_Party_(United_States)'],
['Juan_Perón', 'party', 'Labour_Party_(Argentina)'],
['Ann_Arbor,_Michigan', 'populationDensity', '1580.7 (inhabitants per square kilometre)'],
['1634:_The_Bavarian_Crisis', 'precededBy', 'The_Grantville_Gazettes'],
['Mike_Akhigbe', 'predecessor', 'Oladipo_Diya'],
['GMA_New_Media', 'product', 'World_Wide_Web'],
['Addiction_(journal)', 'publisher', 'Wiley-Blackwell'],
['Arem-arem', 'region', '"Nationwide in Indonesia, but more specific to Java"'],
['Arem-arem', 'region', 'Indonesia'],
['Binignit', 'region', 'Visayas'],
['Abdul_Taib_Mahmud', 'region', 'Kota_Samarahan'],
['AMC_Matador', 'relatedMeanOfTransportation', 'AMC_Ambassador'],
['Alfa_Romeo_164', 'relatedMeanOfTransportation', 'Fiat_Croma'],
['Lancia_Thema', 'relatedMeanOfTransportation', 'Alfa_Romeo_164'],
['Agnes_Kant', 'residence', 'Netherlands'],
['1._FC_Magdeburg', 'season', '2014'],
['A.C._Lumezzane', 'season', '2014–15_Lega_Pro'],
['A.E_Dimitra_Efxeinoupolis', 'season', '2014'],
['A.E_Dimitra_Efxeinoupolis', 'season', '2014–15_A_EPSTH,_Greece'],
['A.S._Roma', 'season', '2014–15_Serie_A'],
['Agremiação_Sportiva_Arapiraquense', 'season', '2015_Campeonato_Brasileiro_Série_C'],
['Bananaman', 'starring', 'Bill_Oddie'],
['Lee_County,_Alabama', 'state', 'Alabama'],
['Potter_County,_Texas', 'state', 'Texas'],
['Jazz', 'stylisticOrigin', 'Blues'],
['Abdul_Taib_Mahmud', 'successor', 'Adenan_Satem'],
['Abdul_Taib_Mahmud', 'successor', 'Sulaiman_Abdul_Rahman_Taib'],
['Abraham_A._Ribicoff', 'successor', 'Anthony_J._Celebrezze'],
['Antonis_Samaras', 'successor', 'Konstantinos_Mitsotakis'],
['1036_Ganymed', 'temperature', '160.0 (kelvins)'],
['109_Felicitas', 'temperature', '170.0 (kelvins)'],
['Alhambra', 'topSpeed', '18.52'],
['Asser_Levy_Public_Baths', 'yearOfConstruction', '1904'],
['Akeem_Priestley', 'youthclub', 'Jacksonville_Dolphins']]

In [ ]:
len(dev_seenTriples), len(good_seenTriples)

(387, 268)

In [ ]:
import random

random.seed(0)

random_seenTriples = random.sample(good_seenTriples, 20)
random_unseenTriples = random.sample(good_unseenTriples, 20)

In [ ]:
random_seenTriples

[['3Arena', 'location', '"East Link Bridge"'],
 ['N._R._Pogson', 'nationality', 'England'],
 ['Wolf_Solent', 'author', 'John_Cowper_Powys'],
 ['Alex_Plante', 'height', '1.9304'],
 ['Abraham_A._Ribicoff', 'successor', 'Anthony_J._Celebrezze'],
 ['Agnes_Kant', 'residence', 'Netherlands'],
 ['1._FC_Köln', 'manager', 'Peter_Stöger'],
 ['Philippines', 'language', 'Arabic'],
 ['Abdul_Taib_Mahmud', 'region', 'Kota_Samarahan'],
 ['Ann_Arbor,_Michigan', 'leaderTitle', '"City Administrator"'],
 ['1634:_The_Ram_Rebellion', 'followedBy', '1635:_The_Cannon_Law'],
 ['Jazz', 'stylisticOrigin', 'Blues'],
 ['Stuart_Parker_(footballer)', 'club', 'Bury_F.C.'],
 ['Angola,_Indiana', 'isPartOf', 'Pleasant_Township,_Steuben_County,_Indiana'],
 ['New_Jersey', 'capital', 'Trenton,_New_Jersey'],
 ['A.S._Gubbio_1910', 'ground', 'Italy'],
 ['200_Public_Square', 'completionDate', '1985'],
 ['Poland', 'language', 'Kashubian_language'],
 ['Premier_Development_League', 'champions', 'K-W_United_FC'],
 ['Adam_Maher', '

In [ ]:
random_unseenTriples

[['Take_It_Off!', 'producer', 'Wharton_Tiers'],
 ['Bootleg_Series_Volume_1:_The_Quine_Tapes', 'releaseDate', '2001-10-16'],
 ['McVeagh_of_the_South_Seas', 'writer', 'Harry_Carey_(actor_born_1878)'],
 ['John_Mills', 'child', 'Juliet_Mills'],
 ['Death_on_a_Factory_Farm', 'producer', 'Tom_Simon'],
 ['Pontiac_Rageous', 'productionEndYear', '1997'],
 ['Take_It_Off!', 'producer', 'The_Honeymoon_Killers_(American_band)'],
 ['Brandon_Carter', 'knownFor', 'Doomsday_argument'],
 ['English_Without_Tears', 'releaseDate', '1944-07-28'],
 ['Live_at_Roadburn_2008_(Year_of_No_Light_album)',
  'recordDate',
  '2008-04-19'],
 ['Liselotte_Grschebina', 'training', 'School_of_Applied_Arts_in_Stuttgart'],
 ["It's_Great_to_Be_Young_(1956_film)", 'musicComposer', 'Louis_Levy'],
 ['Bootleg_Series_Volume_1:_The_Quine_Tapes',
  'artist',
  'The_Velvet_Underground'],
 ['Mermaid_(Train_song)', 'album', 'California_37_(Train_album)'],
 ['Nord_(Year_of_No_Light_album)', 'artist', 'Year_of_No_Light'],
 ['Imagine_(Joh